In [1]:
import hopsworks
import numpy as np
import pandas as pd
import datetime

project = hopsworks.login()
fs = project.get_feature_store()

/home/javierdlrm/.pyenv/versions/jupyter-hopsworks/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2024-09-24 16:18:39,039 INFO: Initializing external client
2024-09-24 16:18:39,040 INFO: Base URL: https://127.0.0.1:28181
2024-09-24 16:18:39,855 INFO: Python Engine initialized.

Logged in to project, explore it here https://127.0.0.1:28181/p/119


In [2]:
trans_fv = fs.get_feature_view(name="trans_fv", version=1)

In [3]:
fmcs = trans_fv.get_feature_monitoring_configs()

In [5]:
# for fmc in fmcs:
#     print("Deleting FMC: " + fmc.name)
#     fmc.delete() 

### FM meta objects

#### FM config, window configs, statistics config, comparison config

In [8]:
fmc = trans_fv.get_feature_monitoring_configs(name="fv_amount_monitoring_reference_td_spec")

In [9]:
def print_fmc(fmc):
    print(fmc.__repr__())
    print(fmc.id)
    print(fmc.name)
    print(fmc.description)
    print(fmc.job_name)
    print(fmc.feature_monitoring_type)
    print(fmc.job_schedule.__repr__())

print_fmc(fmc)


FeatureMonitoringConfig('fv_amount_monitoring_reference_td_spec', STATISTICS_COMPARISON)
16
fv_amount_monitoring_reference_td_spec
Compute and compare descriptive statistics on the amount Feature in the last hour of data inserted in the Feature View
trans_fv_1_run_fm_fv_amount_monitoring_reference_td_spec
STATISTICS_COMPARISON
JobSchedule('0 10,30,50 * ? * * *', enabled: True)


In [10]:
def print_window_config(wc):
    if wc is None: return
    print(wc.__repr__())
    print(wc.id)
    print(wc.window_config_type)
    print(wc.window_length)
    print(wc.time_offset)
    print(wc.training_dataset_version)
    print(wc.row_percentage)

In [11]:
print_window_config(fmc.detection_window_config)

MonitoringWindowConfig(ROLLING_TIME)
20
ROLLING_TIME
None
1h
None
0.2


In [12]:
print_window_config(fmc.reference_window_config)

MonitoringWindowConfig(TRAINING_DATASET)
21
TRAINING_DATASET
None
None
1
1.0


In [13]:
def print_fsc(fsc):
    print(fsc.__repr__())
    print(fsc.id)
    print(fsc.feature_name)
    print(fsc.statistics_comparison_configs)

print_fsc(fmc.feature_statistics_configs[0])

FeatureStatisticsConfig('cc_num')
48
cc_num
[StatisticsComparisonConfig('MAX')]


In [14]:
def print_scc(scc):
    print(scc.__repr__())
    print(scc.id)
    print(scc.metric)
    print(scc.threshold)
    print(scc.relative)
    print(scc.strict)
    print(scc.specific_value)

print_scc(fmc.feature_statistics_configs[0].statistics_comparison_configs[0])

StatisticsComparisonConfig('MAX')
26
MAX
100000000000000.0
False
False
None


#### FM results, statistics results, statistics comparison

In [15]:
fmrs = fmc.get_history()
fmrs

[FeatureMonitoringResult(1727188262000, shift detected: True)]

In [16]:
def print_fmr(fmr):
    print(fmr.__repr__())
    print(fmr.id)
    print(fmr.feature_store_id)
    print(fmr.feature_monitoring_config_id)
    print(fmr.execution_id)
    print(fmr.monitoring_time)
    print(fmr.empty_detection_window)
    print(fmr.empty_reference_window)
    print(fmr.shifted_feature_names)
    print(fmr.shift_detected)
    print(fmr.feature_statistics_results)

print_fmr(fmrs[0])

FeatureMonitoringResult(1727188262000, shift detected: True)
162
67
16
0
1727188262000
False
False
{'amount'}
True
[FeatureStatisticsResult('amount', shift detected: True), FeatureStatisticsResult('cc_num')]


In [17]:
def print_fsr(fsr):
    print(fsr.__repr__())
    print(fsr.id)
    print(fsr.feature_name)
    print(fsr.detection_statistics_id)
    print(fsr.reference_statistics_id)
    print(fsr.shifted_metric_names)
    print(fsr.shift_detected)
    print(fsr.detection_statistics)
    print(fsr.reference_statistics)
    print(fsr.statistics_comparison_results)

print_fsr(fmrs[0].feature_statistics_results[0])

FeatureStatisticsResult('amount', shift detected: True)
556
amount
None
None
{'MEAN'}
True
{"id": 1213, "featureName": "amount", "count": 21411, "featureType": "Fractional", "min": -71.99, "max": 29896.32, "sum": 7722823.479999994, "mean": 360.69419830928, "stdDev": 1495.6075020725157, "completeness": 1.0, "numNonNullValues": 21411, "numNullValues": 0, "distinctness": null, "entropy": null, "uniqueness": null, "approxNumDistinctValues": 10839, "exactNumDistinctValues": null, "percentiles": []}
{"id": 13, "featureName": "amount", "count": 74317, "featureType": "Fractional", "min": 0.01, "max": 29968.32, "sum": 31461210.62000005, "mean": 423.3380063780837, "stdDev": 1456.721404243922, "completeness": 1.0, "numNonNullValues": 74317, "numNullValues": 0, "distinctness": null, "entropy": null, "uniqueness": null, "approxNumDistinctValues": 13713, "exactNumDistinctValues": null, "percentiles": [0.48, 0.94, 1.29, 1.65, 1.95, 3.14, 4.55, 6.12, 7.5, 8.9, 10.12, 10.64, 11.45, 12.29, 13.58, 15.0, 

In [19]:
def print_scr(scr):
    print(scr.__repr__())
    print(scr.id)
    print(scr.feature_statistics_result_id)
    print(scr.statistics_comparison_config_id)
    print(scr.shift_detected)
    print(scr.difference)

print_scr(fmrs[0].feature_statistics_results[0].statistics_comparison_results[0])

StatisticsComparisonResult(shift detected: True)
301
None
27
True
32.9703


### FM with Detection only

### FM with References

---

#### Single feature (specific values)

In [4]:
fv_amount_monitoring_reference_value = trans_fv.create_feature_monitoring(
    name="fv_amount_monitoring_reference_value",
    cron_expression="0 10,30,50 * ? * * *",
    description="Compute and compare descriptive statistics on the amount Feature in the last hour of data inserted in the Feature View",
).with_detection_window(
    time_offset="1h", # fetch data from the last hour
    row_percentage=0.2,
).compare_on(
    feature_name="amount",
    metric="mean",
    threshold=0.1,
    relative=True,
    specific_value=10,
).save()

In [5]:
# fv_amount_monitoring_reference_value.disable()

fv_amount_monitoring_reference_value.run_once()

Job('trans_fv_1_run_fm_fv_amount_monitoring_reference_value', 'PYSPARK')

In [ ]:
fv_amount_monitoring_reference_value = trans_fv.get_feature_monitoring_configs(name="fv_amount_monitoring_reference_value")
# fv_amount_monitoring_reference_value.delete()

#### Subset of features (training dataset version and specific values)

In [6]:
fv_amount_monitoring_reference_td = trans_fv.create_feature_monitoring(
    name="fv_amount_monitoring_reference_td_spec",
    cron_expression="0 10,30,50 * ? * * *",
    description="Compute and compare descriptive statistics on the amount Feature in the last hour of data inserted in the Feature View",
).with_detection_window(
    time_offset="1h", # fetch data from the last hour
    row_percentage=0.2,
).with_reference_training_dataset(
    training_dataset_version=1, # use the training dataset used to train your production model
).compare_on(
    feature_name="amount",
    metric="mean",
    threshold=0.1, # allow for a 10% difference between the two windows before triggering an alert
    relative=True,
).compare_on(
    feature_name="amount",
    metric="mean",
    threshold=0.1,
    relative=True,
    specific_value=10,
).compare_on(
    feature_name="cc_num",
    metric="max",
    threshold=100000000000000,
    relative=False,
).save()

In [ ]:
# fv_amount_monitoring_reference_td.disable()

fv_amount_monitoring_reference_td.run_once()

In [ ]:
fv_amount_monitoring_reference_td = trans_fv.get_feature_monitoring_configs(name="fv_amount_monitoring_reference_td_spec")
# fv_amount_monitoring_reference_td.delete()

#### Subset of features (reference window and specific values)

In [7]:
fv_amount_monitoring_reference_window = trans_fv.create_feature_monitoring(
    name="fv_amount_monitoring_reference_window",
    cron_expression="0 10,30,50 * ? * * *",
    description="Compute and compare descriptive statistics on the amount Feature in the last hour of data inserted in the Feature View",
).with_detection_window(
    time_offset="1h", # fetch data from the last hour
    row_percentage=0.2,
).with_reference_window(
    time_offset="1d3h", # fetch data from the start of same day of the previous week
    window_length="1d", # limit the reference window to the same day of the previous week
    row_percentage=0.8,
).compare_on(
    feature_name="amount",
    metric="mean",
    threshold=0.1, # allow for a 10% difference between the two windows before triggering an alert
    relative=True,
).compare_on(
    feature_name="amount",
    metric="mean",
    threshold=0.1,
    relative=True,
    specific_value=10,
).compare_on(
    feature_name="cc_num",
    metric="max",
    threshold=100000000000000,
    relative=False,
).save()

In [ ]:
# fv_amount_monitoring_reference_window.disable()

fv_amount_monitoring_reference_window.run_once()

In [ ]:
fv_amount_monitoring_reference_window = trans_fv.get_feature_monitoring_configs(name="fv_amount_monitoring_reference_window")
# fv_amount_monitoring_reference_window.delete()

---

In [ ]:
# UNTRANSFORMED FEATURE LOGGING

fv_amount_monitoring_reference_td = trans_fv.create_feature_monitoring(
    name="fv_amount_monitoring_reference_td",
    cron_expression="0 0 12 ? * * *",
    description="Compute and compare descriptive statistics on the amount Feature in the last hour of data inserted in the Feature View",
).with_detection_logging_fg( # untransformed 
    window_length="",
    offset="",
).with_reference_training_dataset(
    training_dataset_version=1, # use the training dataset used to train your production model
).compare_on(
    feature_name="amount",
    metric="mean",
    threshold=50,
).save()